In [1]:
from pynq import Overlay
import numpy as np
import aes as paes
from pynq import allocate
import time
import struct

#data type declaration for streaming, uint_128
puint128 = np.dtype([('upper', np.uint64),('lower', np.uint64)])

#create the stream overlay
stream_overlay=Overlay('/home/xilinx/pynq/overlays/project_aes/aes_opt2.bit')
#uncomment to get info on stream_overlay
#stream_overlay?

#create the direct memory access (dma)
dma = stream_overlay.aes_comb.axi_dma_0
#uncomment to get info on dma
#dma?

In [2]:
#Forward encryption function. Takes in the key and plaintext to encrypt, outputs encrypted text and timing values
#expects the key as a 4-tuple of 32 bits each representing the key
#expects the ptext as a list of 2-part tuples representing 2 64-bit chunks of a 128 bit block
def encrypt(key, ptext):
    dsize = len(ptext)
    #print(dsize)
    d_in = allocate(shape=(dsize,), dtype = puint128)
    d_out = allocate(shape=(dsize,), dtype = puint128)
    for num, data in enumerate(ptext):
        d_in[num][1] = data[0]
        d_in[num][0] = data[1]
    #write the key to the board via axi lite streaming
    start = time.time()
    stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x1c, key[0])
    stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x18, key[1])
    stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x14, key[2])
    stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x10, key[3])
    end = time.time()
    pt1 = end-start
    start = time.time()
    #stream the input to the encryption algorithm and get the output
    dma.sendchannel.transfer(d_in)
    dma.recvchannel.transfer(d_out)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    end = time.time()
    pt2 = end-start
    #return the encrypted data, the time to write the key, and the time to encrypt all the data
    return d_out, pt1, pt2

In [3]:
#file declarations
#file where the key is located, should be 16 characters, or 128 bits
keyFileName = '/home/xilinx/jupyter_notebooks/xilinx/input_files/key.txt'
#file where the plaintext is located, should be a multiple of 16 characters/128 bits (including whitespace)
plaintextFileName = '/home/xilinx/jupyter_notebooks/xilinx/input_files/plaintext.txt'
#hardware output file name, encrypted output from hardware impl will be stored here
eoutputFileName = '/home/xilinx/jupyter_notebooks/xilinx/input_files/hw_encrypted.txt'
#software output file name, encrypted output from hardware impl will be stored here
soutputFileName = '/home/xilinx/jupyter_notebooks/xilinx/input_files/sw_encrypted.txt'


In [4]:
#Software portion
#read files
with open(keyFileName, mode='r') as file:
    skey = file.read()
    file.close()
#print(skey)
#print(type(skey))

with open(plaintextFileName, mode='r') as file:
    sptext = file.read()
    file.close()
#print(sptext)
#print(type(sptext))

#convert inputs to binary
skey_bin = skey.encode('ascii')
sptext_bin = sptext.encode('ascii')

start = time.time()
aes_algo = paes.aes_algo(skey_bin)
sctext = aes_algo.encrypt_bin(sptext_bin)
end = time.time()
print((end - start) * 1000)
#print(sctext)

with open(soutputFileName, mode='wt') as file:
    file.write(bytes.hex(sctext))
    file.close()

1.1930465698242188


In [10]:
#Hardware portion
#read in the files
with open(keyFileName, mode='rb') as file:
    fileContent = file.read()
    #print(fileContent)
    #read in the file (interpreted as binary), 4 Long ints
    keybinary = struct.unpack(">LLLL", fileContent)
    file.close()

with open(plaintextFileName, mode='rb') as file:
    fileContent = file.read()
    #read in the file in portions of 2 64-bit ints
    plaintextBinary = struct.iter_unpack(">QQ", fileContent)
    file.close()

ktext = list(keybinary)
ptext = list(plaintextBinary)
"""
print("cipher key in hex: " + hex(keybinary[0]) + " " + hex(keybinary[1]) + " " + hex(keybinary[2]) + " " + hex(keybinary[3]))

print("plaintext in 128 bit blocks in hex:")
for a in ptext:
    print(hex(a[0]) + " " + hex(a[1]))
""" 
print(len(ptext))
d_out = allocate(shape=(len(ptext),), dtype = puint128)
d_out, t1, t2 = encrypt(ktext, ptext)

# get the encrypted binary, and decrypt with software
ebin = bytearray()
for a in d_out:
    for i in struct.pack('>Q', a[1]):
        ebin.append(i)
    for i in struct.pack('>Q', a[0]):
        ebin.append(i)
    

#print(ebin)
#print(bytes(ebin))
print("decrypted text")
print(aes_algo.decrypt_bin(bytes(ebin)))


#print out the timeing
print("Time to write key to board: " + str(t1*1000))
print("Time to stream I/O and encrypt: " + str(t2*1000))
#print(d_out)

#open the output file for writing binary as hex
with open(eoutputFileName, mode='wt') as file:
    for e in d_out:
        a = int(e[1])
        b = int(e[0])
        s = a << 64 | b
        #print(type(s))
        #convert to hex and write to the file
        file.write(format(s, 'x'))
    file.close()

1
decrypted text
Two One Nine Two
Time to write key to board: 80.0633430480957
Time to stream I/O and encrypt: 0.9388923645019531


In [6]:
#if the below returns nothing, then the encryptions are the same, otherwise they are different
!diff /home/xilinx/jupyter_notebooks/xilinx/input_files/hw_encrypted.txt /home/xilinx/jupyter_notebooks/xilinx/input_files/sw_encrypted.txt